In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# Path to the dataset
data_path = r'C:\Users\kores\OneDrive\Desktop\Ngishu\cattle_face_images_dataset'

# Parameters
img_height, img_width = 128, 128
batch_size = 20

# Function to load images and labels
def load_data(data_path):
    images = []
    labels = []
    label_map = {}
    current_label = 0

    for label_folder in os.listdir(data_path):
        label_folder_path = os.path.join(data_path, label_folder)
        if os.path.isdir(label_folder_path):
            label_map[current_label] = label_folder
            for img_file in os.listdir(label_folder_path):
                img_path = os.path.join(label_folder_path, img_file)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (img_height, img_width))
                    images.append(img)
                    labels.append(current_label)
            current_label += 1

    return np.array(images), np.array(labels), label_map

# Load data
images, labels, label_map = load_data(data_path)

# Normalize the images
images = images / 255.0

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=52)


In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the ResNet50 model, excluding the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(label_map), activation='softmax')(x)

# Combine the base model with the custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [4]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fit the model
history = model.fit(x_train, y_train,
                    epochs=50,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.0318 - loss: 4.0096 - val_accuracy: 0.0677 - val_loss: 3.8298
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.0870 - loss: 3.7275 - val_accuracy: 0.1474 - val_loss: 3.6201
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.1508 - loss: 3.5111 - val_accuracy: 0.1594 - val_loss: 3.4051
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.1995 - loss: 3.3434 - val_accuracy: 0.2271 - val_loss: 3.2017
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.2699 - loss: 3.0830 - val_accuracy: 0.3466 - val_loss: 2.9774
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.2854 - loss: 2.9309 - val_accuracy: 0.3068 - val_loss: 2.8070
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.3747 - loss: 2.7311 - val_accuracy: 0.3506 - val_loss: 2.6191
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.4123 - loss: 2.4912 - val_accuracy: 0.3865 - val_loss:

In [5]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.2f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9180 - loss: 0.3305
Test accuracy: 0.90


In [7]:
model.save('cattle_model5.keras')
